In [ ]:
# install all the requirements for miniCPM-V
!pip install git+https://github.com/huggingface/transformers
!pip install --upgrade modelscope sentencepiece accelerate bitsandbytes datamodel_code_generator

In [ ]:
!pip install deepspeed>=0.9.3
!pip install liger-kernel-nightly==0.2.1.dev20240911164559

In [ ]:
!git clone https://github.com/modelscope/swift.git
%cd swift
!pip install -e '.[llm]'

In [ ]:
!pip install pyav

In [ ]:
!pip install qwen_vl_utils

## QWEN-VL-2B DATASET CREATION

In [ ]:
#Create Stratified Sample to Train Data for Training VLM
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the CSV file
df = pd.read_csv('/kaggle/input/amazon-ml-train/train.csv')

# Calculate the sample size (1% of total rows)
sample_size = int(len(df) * 0.01)

# Perform stratified sampling
stratified_sample, _ = train_test_split(
    df,
    test_size=1 - (sample_size / len(df)),
    stratify=df['entity_name'],
    random_state=42  # for reproducibility
)

# Verify the sample size
print(f"Original dataset size: {len(df)}")
print(f"Sampled dataset size: {len(stratified_sample)}")

# Verify the distribution
print("\nOriginal distribution:")
print(df['entity_name'].value_counts(normalize=True))
print("\nSampled distribution:")
print(stratified_sample['entity_name'].value_counts(normalize=True))

# Save the sampled dataset
stratified_sample.to_csv('/kaggle/working/sample_train.csv', index=False)
print("\nSampled dataset saved as 'sample_train.csv'")

In [ ]:
%cd ..

In [ ]:
# download images
import os
from time import time
import multiprocessing
import logging
import requests
import pandas as pd
import time as t
import random 

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

img_dir = 'image_dir'

def get_sbu_urls():
    df = pd.read_csv('/kaggle/working/sample_train.csv')
    urls = df['image_link'].tolist()
    indices = df.index
    return list(zip(indices, urls))

def scrape_and_save(args):
    url, savepath = args
    retries = 3
    for attempt in range(retries):
        try:
            session = requests.Session()
            response = session.get(url, timeout=1)
            response.raise_for_status()
            with open(savepath, 'wb') as f:
                f.write(response.content)
            logging.info(f"Successfully downloaded: {url}")
            return True
        except requests.RequestException as e:
            logging.error(f"Error downloading {url}: {e} (Attempt {attempt+1}/{retries})")
        except IOError as e:
            logging.error(f"Error saving file {savepath}: {e}")
            return False
        t.sleep(2)  # small delay before retrying
    return False

if __name__ == '__main__':
    startidx = 0
    chunk_size = 5000  # Break downloads into chunks
    urls = get_sbu_urls()
    total_urls = len(urls)

    if not os.path.exists(img_dir):
        os.makedirs(img_dir)

    starttime = time()
    piccounter = 0

    # Download in chunks
    for chunk_start in range(startidx, total_urls, chunk_size):
        chunk_end = min(chunk_start + chunk_size, total_urls)
        url_chunk = urls[chunk_start:chunk_end]
        
        pool = multiprocessing.Pool(16)  # create a new pool for each chunk

        results = []
        for i, (index, url) in enumerate(url_chunk, start=chunk_start):
            name = f'train_{index}.jpg'
            savepath = os.path.join(img_dir, name)
            result = pool.apply_async(scrape_and_save, ((url, savepath),))
            results.append(result)

        pool.close()  # no more tasks
        pool.join()  # wait for completion

        # Count successful downloads in the chunk
        successful_downloads = sum([r.get() for r in results])
        piccounter += successful_downloads

        logging.info(f"Downloaded {successful_downloads}/{chunk_size} in chunk {chunk_start//chunk_size + 1}. Total downloaded: {piccounter}/{total_urls}")

        # Throttle requests to prevent server throttling (optional)
        t.sleep(10)  # 10-second break between chunks

    print(f"Downloaded {piccounter}/{total_urls} images.")
    print(f"Total time taken: {time() - starttime:.2f} seconds")

In [ ]:
import pandas as pd
import json
import os

entity_unit_map = {
    'depth': ['mm',"'",'cm','centimetre', 'millimetre', 'inch', 'metre', 'foot'],
    'height': ['centimetre', 'millimetre', 'inch', 'metre', 'foot'],
    'width': ['millimetre', 'centimetre', 'inch', 'foot', 'metre'],
    'item_volume': ['cup', 'gallon', 'ounce', 'millilitre', 'cubic foot', 'fluid ounce', 'decilitre', 'cubic inch', 'litre', 'quart', 'pint', 'centilitre'],
    'item_weight': ['gram', 'milligram', 'kilogram', 'kilogram to kilogram', 'ounce', 'pound', 'ton', 'microgram', 'millilitre', 'gigabyte', 'gram to gram', 'kilogram to ilogram', 'candela', 'fluid ounce', 'ounce to ounce', 'pound to pound', 'cup', 'milligram to milligram', 'carat', 'litre', 'e pound', 'nits', 'inch', 'quart', 'watt', 'millimetre', 'centimetre', 'millilitre to millilitre', 'cubic inch'],
    'maximum_weight_recommendation': ['kilogram', 'pound', 'ounce', 'ton', 'milligram', 'kilogram to kilogram', 'gram', 'microgram', 'person', 'pound to pound', 'pound to ound', 'foot', 'kilogram to ilogram', 'horsepower'],
    'voltage': ['volt'],
    'wattage': ['watt', 'horsepower', 'kilowatt', 'kilowatt hour', 'milliampere hour']
}


def prepare_dataset(csv_path, image_dir, output_json_path, question, answer_column):
    # Read the CSV file
    df = pd.read_csv(csv_path)
    
    # Prepare the data in the required format
    data = []
    for index, row in df.iterrows():
        image_path = os.path.join(image_dir, f"train_{index}.jpg")
        
        # Check if the image file exists
        if not os.path.exists(image_path):
            print(f"Warning: Image file not found for index {index}")
            continue
        
        entry = {
            "system": f"You are an entity extractor OCR model. Given the entity, you can extract the text from the image denoting the entity value. Entity value is always a number followed by its unit like {entity_unit_map[row['entity_name']]}.",
            "query": f"<image>What is the value of the {row['entity_name']} of the item shown? Give me numerical value and unit as seen in the image. Example Input: What is the value of the volume of the item shown, Output : 1 cup. Adhere to instructions.",
            "response": f"{row['entity_value'].split()}",
            "images": [f"{image_path}"]
        }
        data.append(entry)
    
    # Write the data to a JSON file
    with open(output_json_path, 'w') as f:
        json.dump(data, f, indent=2)
    
    print(f"Dataset prepared and saved to {output_json_path}")

# Usage
csv_path = '/kaggle/working/sample_train.csv'
image_dir = '/kaggle/working/image_dir/'
output_json_path = '/kaggle/working/train.json'

prepare_dataset(csv_path, image_dir, output_json_path, question, answer_column)

In [ ]:
!SIZE_FACTOR=8 MAX_PIXELS=313600 CUDA_VISIBLE_DEVICES=0,1 swift sft \
    --model_type qwen2-vl-2b-instruct \
    --model_id_or_path qwen/Qwen2-VL-2B-Instruct \
    --sft_type lora \
    --dataset /kaggle/working/train.json \
    --freeze_vit \
    --seed 42 \
    --max_length 128 \
    --lora_rank 32 \
    --use_liger \
    --num_train_epochs 1

In [ ]:
!zip -r checkpoint163.zip /kaggle/working/output/qwen2-vl-2b-instruct/v3-20240914-130231/checkpoint-163/

In [ ]:
!ls -lah